# BHT Data Applications project
# Automatic Anime recommendation Algorithm
### This project aims to create an algorithm that can determine what anime to recommend to a user.
##### Authors: Rashmi Di Michino and Antonin Mathubert

The 320000 users and 16000 animes dataset was taken from https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020 <br>
We are going to use this dataset to build a model that can recommend an anime based on the animes that the user is watching, has dropped, has kept on hold or put on their watching list.

### 1. Importing and parsing the data
First, we want to import all of our available data in a suitable manner so it is treatable for the next steps of the project.<br><br>
Here, we are going to propose two methods to do that. The first one is very slow but the "clean" way to read json data. Because we don't actually need all the data from the files, we are going to use a second hand made method that is significantly faster.

In [1]:
from mlxtend.frequent_patterns import apriori, association_rules
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import cupy as cp
import random
import time
import os
import re

In [2]:
dataset_chunks = pd.read_csv("dataset/anime/animelist.csv", chunksize=10000)
chunks = []
for chunk in dataset_chunks:
    chunks.append(chunk)
    
chunks = random.sample(chunks, int(len(chunks) / 3))

dataset = pd.concat(chunks, ignore_index=True)
dataset = dataset.astype({'user_id': "int32", 'anime_id': 'int32', "watching_status": "int16"})

dataset_chunks = None
chunks = None

In [ ]:
dataset_chunks = pd.read_csv("C:/Users/rashm/OneDrive/Desktop/data_applications_project/julius/anime_dataset/animelist.csv", chunksize=10000)

chunks = []
for chunk in dataset_chunks:
    chunks.append(chunk)
    
dataset = pd.concat(chunks, ignore_index=True)
dataset = dataset.astype({'user_id': "int32", 'anime_id': 'int32', "watching_status": "int16"})

In [3]:
dataset.drop(['rating', 'watched_episodes'], axis=1, inplace=True)
dataset = dataset[(dataset['anime_id'] < 10000) & dataset['user_id'] < 100000]
dataset = dataset[(dataset['user_id'] != 61960) & (dataset['watching_status'] != 4)]

In [4]:
display(dataset.head(100))
len(dataset)

,user_id,anime_id,watching_status
0,95000,33255,6
1,95000,205,6
2,95000,31229,6
3,95000,25777,6
4,95000,25835,6
...,...,...,...
95,95001,24655,2
96,95001,14353,2
97,95001,1535,2
98,95001,28223,2


34981073

In [5]:
dataset = dataset.drop("watching_status", axis=1)

In [6]:
dataset = dataset.pivot(index='user_id', columns='anime_id', values='anime_id')

In [8]:
dataset = np.array(dataset.values)

In [10]:
dataset = np.nan_to_num(dataset, nan=0)

In [11]:
dataset = np.where(dataset != 0, 1, dataset)

In [ ]:
dataset[dataset.notnull()] = True

In [ ]:
dataset = dataset.fillna(False)

In [12]:
frequent_itemsets  = apriori(dataset, use_colnames=True, min_support=0.1)

frequent_itemsets

AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'

In [ ]:
rules = association_rules(frequent_itemsets)

rules

In [ ]:
rules[rules["antecedents"]==frozenset({1535})]